In [2]:
import pandas as pd
from datetime import datetime, timedelta


DICT_PRODUCCION = {
    "Medico Quirugico 3° Norte y Sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "Medico Quirugico 4° Norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "MQ 3° NORTE": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "MQ 3° SUR": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "MQ 3° Norte": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "MQ 3° Norte\t": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "MQ 3° Sur": "HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)",
    "MQ 4° Norte": "HOSPITALIZACIÓN CARDIOVASCULAR",
    "UCI 4° Sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "UCI 5° Norte": "UNIDAD DE CUIDADOS INTENSIVOS",
    "UCI 5° NORTE": "UNIDAD DE CUIDADOS INTENSIVOS",
    "UCI 5° Sur": "UNIDAD DE CUIDADOS INTENSIVOS",
    "UTI 3° Norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "UTI 4° Norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "UTI 5° Norte": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "UTI 4° Sur": "UNIDAD DE TRATAMIENTO INTENSIVO ADULTO",
    "Ambulatorio": "CONSULTA",
    "AMBULATORIA": "CONSULTA",
    "AMBULATORIO": "CONSULTA",
}

HOJA_PRODUCCION_TIPO_PRODUCCION = {
    "QMT": "PROCEDIMIENTO ONCOLOGÍA",
    "APNEA": "PROCEDIMIENTO DE NEUMOLOGÍA",
    "EF": "PROCEDIMIENTOS DE CARDIOLOGÍA",
    "EBUS": "PROCEDIMIENTOS EBUS",
    "TAVI": "PROCEDIMIENTOS TAVI",
    "ASIST VENTRICULAR": "PROCEDIMIENTOS ASISTENCIA VENTRICULAR",
    "ECMO": "PROCEDIMIENTOS ECMO",
    "HMD": "PROCEDIMIENTOS DE HEMODINAMIA",
    "BRONCOSCOPIA": "PROCEDIMIENTOS ENDOSCÓPICOS",
    "Anatomia Patologica": "ANATOMÍA PATOLÓGICA",
    "Imagenologia": "IMAGENOLOGÍA",
    "Tomografia": "TOMOGRAFÍA",
}

HOMOLOGIA_CONSULTA = {
    "PROCEDIMIENTO ONCOLOGÍA": "CONSULTA ONCOLOGÍA",
    "PROCEDIMIENTO DE NEUMOLOGÍA": "CONSULTA NEUMOLOGÍA",
    "PROCEDIMIENTOS DE CARDIOLOGÍA": "CONSULTA CARDIOLOGÍA",
    "PROCEDIMIENTOS EBUS": "CONSULTA CARDIOLOGÍA",
    "PROCEDIMIENTOS TAVI": "",
    "PROCEDIMIENTOS ASISTENCIA VENTRICULAR": "",
    "PROCEDIMIENTOS ECMO": "",
    "PROCEDIMIENTOS DE HEMODINAMIA": "CONSULTA CARDIOLOGÍA",
    "PROCEDIMIENTOS ENDOSCÓPICOS": "CONSULTA NEUMOLOGÍA",
    "ANATOMÍA PATOLÓGICA": "",
    "IMAGENOLOGÍA": "",
    "TOMOGRAFÍA": "",
}

def get_first_and_last_day(year, month):
    first_day = datetime(year, month, 1)
    next_month = first_day.replace(month=first_day.month + 1, day=1)
    last_day = next_month - timedelta(days=1)
    last_day = last_day.replace(hour=23, minute=59, second=59)
    return first_day, last_day

In [5]:
archivo_produccion = pd.ExcelFile("../data/raw/produccion_julio_2023.xlsx")

In [6]:
INICIO_HOJA_PRODUCCIONES = 3
COLUMNAS_LOCALIZACION_PROCEDIMIENTO = ["Tipo de Atencíon", "Hoja1.Nombre nuevo HIS"]



anio_a_analizar = int(input("> Qué año quieres analizar? (Ingresa solamente el año en números): )"))
mes_a_analizar = int(input("> Qué mes quieres analizar? (Ingresa solamente el numero del mes):"))

ANIO_A_ANALIZAR = 2023
MES_A_ANALIZAR = 7


hojas_producciones = archivo_produccion.sheet_names[INICIO_HOJA_PRODUCCIONES:]
primera_fecha, ultima_fecha = get_first_and_last_day(ANIO_A_ANALIZAR, MES_A_ANALIZAR)

In [7]:
ultima_fecha

datetime.datetime(2023, 7, 31, 23, 59, 59)

In [21]:
resultado = {}
for hoja in hojas_producciones:
    df = pd.read_excel(archivo_produccion, sheet_name=hoja)
    df["Fecha Realización"] = pd.to_datetime(df["Fecha Realización"], dayfirst=True)
    df_mes = df.query(
        "`Fecha Realización` >= @primera_fecha and `Fecha Realización` <= @ultima_fecha"
    ).copy()
    if hoja == "EF":
        electrofis = df.copy()
    tipo_produccion_sigcom = HOJA_PRODUCCION_TIPO_PRODUCCION[hoja]

    print(f"> {hoja} - {tipo_produccion_sigcom}: {df_mes.shape[0]}")

    try:
        df_mes["SIGCOM"] = df_mes["Tipo de Atencíon"].replace(DICT_PRODUCCION)

    except KeyError:
        df_mes["SIGCOM"] = df_mes["Hoja1.Nombre nuevo HIS"].replace(DICT_PRODUCCION)

    conteo_sigcom = df_mes["SIGCOM"].value_counts()

    print(conteo_sigcom, "\n")

    resultado[tipo_produccion_sigcom] = conteo_sigcom

> QMT - PROCEDIMIENTO ONCOLOGÍA: 134
CONSULTA    134
Name: SIGCOM, dtype: int64 

> APNEA - PROCEDIMIENTO DE NEUMOLOGÍA: 57
CONSULTA                                                          55
HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)     2
Name: SIGCOM, dtype: int64 

> EF - PROCEDIMIENTOS DE CARDIOLOGÍA: 18
HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)    8
CONSULTA                                                          6
UNIDAD DE TRATAMIENTO INTENSIVO ADULTO                            3
UNIDAD DE CUIDADOS INTENSIVOS                                     1
Name: SIGCOM, dtype: int64 

> EBUS - PROCEDIMIENTOS EBUS: 23
CONSULTA                                                          20
UNIDAD DE TRATAMIENTO INTENSIVO ADULTO                             2
HOSPITALIZACIÓN MEDICINA INTERNA/HOSPITALIZACION QUIRUGICA (%)     1
Name: SIGCOM, dtype: int64 

> TAVI - PROCEDIMIENTOS TAVI: 7
UNIDAD DE CUIDADOS INTENSIVOS    7
Name: SIGCOM, dtype: int64 

> 

In [22]:
formato_produccion = pd.read_excel(
    "input/Formato_5_Distribución_Producción_CC_Apoyo.xlsx", header=2
).set_index("Centro de Costo")

formato_a_rellenar = formato_produccion.copy()

In [23]:
for tipo_produccion, df_resultado in resultado.items():
    for cc_sigcom, cantidad_procedimientos in df_resultado.items():
        if cc_sigcom == "CONSULTA":
            cc_sigcom = HOMOLOGIA_CONSULTA[tipo_produccion]

        formato_a_rellenar.loc[tipo_produccion, cc_sigcom] = cantidad_procedimientos

In [24]:
formato_a_rellenar.to_excel("prueba.xlsx")